## Week 5 Lab Manual
### Foundations of Deep Learning & AI Functionality

**Instructor Note**: This lab manual provides the aim, code, and explanation for each practical task. Focus on the architectural patterns and the transition from theoretical concepts to functional AI implementations.

---

# Week 5: RAG Systems & Semantic Search

Welcome to Week 5! This week we dive into **Retrieval Augmented Generation (RAG)**. Instead of just asking an LLM to "hallucinate" an answer, we provide it with specific, relevant facts from a database to ensure accuracy and reduce errors.

###  Weekly Table of Contents
1. [Basic Context Injection (Manual RAG mapping)](#-Lab-5.1:-Basic-Context-Injection)
2. [Automated Document Loading & Splitting](#-Lab-5.2:-Automated-Document-Loading-&-Splitting)
3. [Vector Databases (Chroma) & Semantic Search](#-Lab-5.3:-Vector-Databases-(Chroma)-&-Semantic-Search)
4. [Building a Conversational Retrieval Chain](#-Lab-5.4:-Building-a-Conversational-Retrieval-Chain)

###  Learning Objectives
*   **Context Injection**: Learn how to manually feed data into LLMs.
*   **Vector Embeddings**: Understand how text is converted into numbers (vectors).
*   **Vector Databases**: Use **ChromaDB** and **FAISS** to store and retrieve information.
*   **Retrieval Chains**: Build conversational systems that remember history and use external knowledge.
*   **Advanced RAG**: Optimize chunking, metadata, and retrieval parameters.

---

###  Configuration & API Setup
We'll start by ensuring our environment is ready. We use Gemini 1.5 Flash as our primary cloud model and Gemma 2:2b for local tasks.

##  Lab 5.1: Basic Context Injection
**Aim**: To understand the fundamental logic of "Augmentation" by building a manual retrieval system that injects text from local files into a prompt.

**Explanation**:
This lab breaks down the "R" in RAG:
1.  **Retrieval**: We search a dictionary representing a knowledge base for keywords.
2.  **Augmentation**: The retrieved text is prepended to the user query as "Context".
3.  **Generation**: Gemini 1.5 Flash uses this context to answer questions it wasn't originally trained on.

In [ ]:
# 📦 WEEK 5 INITIALIZATION
import os
import glob
import shutil
import google.generativeai as genai
import ollama
from dotenv import load_dotenv
from IPython.display import Markdown, display
import numpy as np
import plotly.graph_objects as go
from sklearn.manifold import TSNE
import gradio as gr

# RAG & LANGCHAIN LIBS
from langchain_google_genai import ChatGoogleGenerativeAI, GoogleGenerativeAIEmbeddings
from langchain_chroma import Chroma
from langchain_community.document_loaders import PyPDFLoader, DirectoryLoader, TextLoader
from langchain_text_splitters import RecursiveCharacterTextSplitter, CharacterTextSplitter
from langchain.memory import ConversationBufferMemory
from langchain.chains import ConversationalRetrievalChain
from langchain_core.callbacks import StdOutCallbackHandler

# --- CONFIGURATION ---
load_dotenv(override=True)
MODEL = "gemini-1.5-flash"
LOCAL_MODEL = "gemma2:2b"

# Ensure API Key is available
GEMINI_API_KEY = os.getenv("GOOGLE_API_KEY") or os.getenv("GEMINI_API_KEY")
if GEMINI_API_KEY:
    os.environ["GOOGLE_API_KEY"] = GEMINI_API_KEY
    genai.configure(api_key=GEMINI_API_KEY)
else:
    print("⚠️ WARNING: GEMINI_API_KEY not found.")

# Create folders for knowledge base
for folder in ["knowledge-base/products", "knowledge-base/employees", "knowledge-base/contracts", "knowledge-base/company"]:
    os.makedirs(folder, exist_ok=True)

print(f"✅ Week 5 Ready: Using {MODEL}")


In [ ]:
# Define content for our documents
kb_files = [
    # Products
    ("knowledge-base/products/carllm.md", "# CarLLM\nCarLLM is our premium AI-powered auto insurance tool. It provides real-time risk assessment and reduced premiums for safe drivers."),
    ("knowledge-base/products/homellm.md", "# HomeLLM\nHomeLLM automates home insurance claims processing using computer vision to assess property damage from photos."),
    
    # Employees
    ("knowledge-base/employees/ceo.md", "# Alex Lancaster\nAlex Lancaster is the CEO and founder of Insurellm. He previously led innovation at several Global 500 insurers."),
    ("knowledge-base/employees/cto.md", "# Sarah Avery\nSarah Avery is the CTO. She is a pioneer in Generative AI for finance and holds 10 patents in neural network optimization."),
    
    # Company Info
    ("knowledge-base/company/about.md", "# About Insurellm\nFounded in 2018, Insurellm is headquartered in London. We are the first cloud-native insurance provider built entirely on Large Language Models."),
    ("knowledge-base/company/awards.md", "# Awards\nInsurellm won the 'Insurance Innovation of the Year' (IIOTY) in 2023 for its work on CarLLM.")
]

# Write the files
for filepath, content in kb_files:
    with open(filepath, 'w', encoding='utf-8') as f:
        f.write(content)

print(f"✅ Created {len(kb_files)} knowledge base files in the 'knowledge-base/' directory.")


### Load Knowledge Base - Employees
Loading employee data from markdown files into a dictionary for simple keyword-based context retrieval.

In [ ]:
# 1. PRE-DEFINED CONTEXT DICTIONARY
context = {
    'Lancaster': 'Alex Lancaster is the CEO of Insurellm. He founded the company in 2018 and has 15 years of experience in insurance technology.',
    'Avery': 'Sarah Avery is the CTO of Insurellm. She leads the technical team and oversees all product development.',
    'CarLLM': "CarLLM is Insurellm's flagship AI-powered car insurance product. It won the prestigious IIOTY (Insurance Innovation of the Year) award in 2023.",
    'HomeLLM': 'HomeLLM is our home insurance AI assistant that helps customers understand coverage and file claims efficiently.',
    'HealthLLM': "HealthLLM is a new project in the R&D stage, focused on personal health insurance tracking.",
    'PolicyLLM': "PolicyLLM helps users understand the jargon in their existing insurance policies using natural language processing."
}

# 2. CONTEXT RETRIEVAL LOGIC
def get_relevant_context(message):
    """Simple keyword-based context retriever"""
    relevant_context = []
    message_lower = message.lower()
    for key, text in context.items():
        if key.lower() in message_lower:
            relevant_context.append(text)
    return relevant_context

def add_context(message):
    """Augments user message with retrieved context"""
    relevant_context = get_relevant_context(message)
    if relevant_context:
        augmented = "The following additional context might be relevant:\n\n"
        for fact in relevant_context:
            augmented += f"- {fact}\n"
        return f"{augmented}\nQuestion: {message}"
    return message

# 3. CHAT GENERATION
system_message = "You are an expert on Insurellm. Give brief, accurate answers based ONLY on provided context."

def chat(message, history):
    prompt = add_context(message)
    model = genai.GenerativeModel(model_name=MODEL, system_instruction=system_message)
    response = model.generate_content(prompt)
    return response.text

print("✅ Manual RAG functions ready.")


### Gemini Chat Function
Main chat function that processes user messages, adds context, and generates responses using Gemini 1.5 Flash.

### Lab 5.1: Gradio Chat Interface
A simple RAG prototype using keyword-based context matching with Gemini 1.5 Flash.

In [ ]:
# Launch Day 1 Gradio interface
view = gr.ChatInterface(chat, type="messages").launch()

##  Lab 5.2: Automated Document Loading & Splitting

**Aim**: To automate the ingestion and transformation of raw document folders into manageable text chunks for RAG pipelines.

**Explanation**: Manual context injection doesn't scale. In this lab, we use LangChain to load a directory of markdown files and split them into chunks using the `RecursiveCharacterTextSplitter` to maintain semantic context within sub-documents.

### Document Loading with LangChain
Loading documents from our knowledge base using LangChain's DirectoryLoader with proper metadata.

In [ ]:
# 1. DOCUMENT LOADING
loader = DirectoryLoader("knowledge-base", glob="**/*.md", loader_cls=TextLoader)
documents = loader.load()

# Assign metadata for categorization
for doc in documents:
    path_parts = doc.metadata['source'].split(os.sep)
    if len(path_parts) >= 2:
        doc.metadata['doc_type'] = path_parts[-2]
    else:
        doc.metadata['doc_type'] = 'general'

# 2. TEXT SPLITTING (RecursiveCharacterTextSplitter)
text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=1000,
    chunk_overlap=100,
    separators=["\n\n", "\n", " ", ""]
)
chunks = text_splitter.split_documents(documents)

print(f"✅ Loaded {len(documents)} documents.")
print(f"✅ Split into {len(chunks)} chunks.")
if chunks:
    print(f"Sample chunk types: {set(c.metadata['doc_type'] for c in chunks)}")


##  Lab 5.3: Vector Databases (Chroma) & Semantic Search

**Aim**: To implement persistent vector storage and semantic retrieval using ChromaDB and Google Generative AI Embeddings.

**Explanation**: We store processed chunks in a vector database to perform semantic search instead of simple keyword matching. This allows the AI to find relevant context based on 'meaning' rather than just exact word matches.

In [ ]:
# 1. EMBEDDINGS SETUP
embeddings = GoogleGenerativeAIEmbeddings(model="models/embedding-001")

# 2. VECTOR DATABASE (CHROMA)
db_path = "insurellm_vector_db"

if os.path.exists(db_path):
    shutil.rmtree(db_path)

vectorstore = Chroma.from_documents(
    documents=chunks, 
    embedding=embeddings, 
    persist_directory=db_path
)

print(f"✅ Vectorstore created with {len(chunks)} chunks using {embeddings.model}.")


In [ ]:
# Get one vector and find how many dimensions it has
collection = vectorstore._collection
sample_embedding = collection.get(limit=1, include=["embeddings"])["embeddings"][0]
dimensions = len(sample_embedding)
print(f"The vectors have {dimensions:,} dimensions")

##  Lab 5.4: Building a Conversational Retrieval Chain

**Aim**: To assemble a complete, stateful Retrieval-Augmented Generation (RAG) system with conversational memory.

**Explanation**: In this final lab, we combine document splitting, vector storage, and memory. The `ConversationalRetrievalChain` manages the flow: searching for context, merging with history, and generating informed responses.

In [ ]:
# 1. SETUP LLM & RETRIEVER
llm = ChatGoogleGenerativeAI(temperature=0.7, model=MODEL)
retriever = vectorstore.as_retriever(search_kwargs={"k": 3})

# 2. SETUP CONVERSATION MEMORY
memory = ConversationBufferMemory(memory_key='chat_history', return_messages=True)

# 3. CREATE CONVERSATIONAL RETRIEVAL CHAIN
rag_chain = ConversationalRetrievalChain.from_llm(
    llm=llm, 
    retriever=retriever, 
    memory=memory,
    verbose=False
)

def chat_rag(message, history):
    # This function is used by Gradio to maintain its own history if needed,
    # but the chain's memory also tracks it.
    result = rag_chain.invoke({"question": message})
    return result["answer"]

print(f"✅ Conversational RAG Chain ready with {MODEL}.")


In [ ]:
# Launch the final Conversational RAG interface
view_rag = gr.ChatInterface(chat_rag, type="messages").launch(inbrowser=True)


---

##  Instructor's Evaluation & Lab Summary

###  Assessment Criteria
1. **Technical Implementation**: Adherence to the lab objectives and code functionality.
2. **Logic & Reasoning**: Clarity in the explanation of the underlying AI principles.
3. **Best Practices**: Use of secure environment variables and structured prompts.

**Lab Completion Status: Verified**
**Focus Area**: Language Modelling & Deep Learning Systems.